In [31]:
df = pd.read_csv("calls_weather_pubhols_25_04_2023.csv")

In [32]:
import pandas as pd
from tqdm import tqdm

# tqdm.pandas() is a decorator to apply progress bars on Pandas groupby apply operations
tqdm.pandas()

# Convert the Date column to a datetime object
df['date'] = pd.to_datetime(df['date'])

# Set the index to be the datetime object
df.set_index('date', inplace=True)

# Sort the DataFrame by Station Area and Date
df.sort_values(by=['Station Area', 'date'], inplace=True)

# Calculate the rolling sum of the count of calls for each group within the past 1 hour, 3 hours, and 6 hours
for window, col_name in zip([1, 3, 6], ['Count_in_last_1hour', 'Count_in_last_3hours', 'Count_in_last_6hours']):
    df[col_name] = df.groupby('Station Area')['Count'].progress_apply(lambda x: x.rolling(window=f'{window}H', closed='right').sum().shift()).fillna(0)

# Group the data by Station Area, date, and hour
hourly_sums = df.groupby(['Station Area', df.index.date, df.index.hour])['Count'].sum().reset_index(name='Hourly_Count')

# Rename columns in the hourly_sums DataFrame for clarity
hourly_sums.columns = ['Station Area', 'Date', 'Hour', 'Hourly_Count']

# Calculate the rolling sum of the count of calls for each group within the past 7 days, 14 days, and 28 days for the same hour
for window, col_name in zip([7, 14, 28], ['Count_in_last_7days', 'Count_in_last_14days', 'Count_in_last_28days']):
    hourly_sums[col_name] = hourly_sums.groupby(['Station Area', 'Hour'])['Hourly_Count'].progress_apply(lambda x: x.rolling(window=window).sum()).fillna(0)

# Merge the hourly sums DataFrame with the original DataFrame
df = df.merge(hourly_sums.drop(columns=['Hourly_Count']), left_on=['Station Area', df.index.date, df.index.hour], right_on=['Station Area', 'Date', 'Hour'], how='left')

# Reset the index back to a regular column
df.reset_index(inplace=True)

# Convert the new columns to integer type
for col_name in ['Count_in_last_1hour', 'Count_in_last_3hours', 'Count_in_last_6hours', 'Count_in_last_7days', 'Count_in_last_14days', 'Count_in_last_28days']:
    df[col_name] = df[col_name].astype('int')

df.head()



  0%|                                                                                           | 0/15 [00:00<?, ?it/s]C:\Users\CiaranJones\anaconda3.1\lib\site-packages\tqdm\std.py:814: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  return getattr(df, df_function)(wrapper, **kwargs)
  0%|                                                                                           | 0/15 [00:00<?, ?it/s]C:\Users\CiaranJones\anaconda3.1\lib\site-packages\tqdm\std.py:814: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the appl

,index,Station Area,Date,Date_x,Time Of Call,Count,rain,temp,wetb,dewpt,...,season,Public Holiday,Count_in_last_1hour,Count_in_last_3hours,Count_in_last_6hours,Date_y,Hour,Count_in_last_7days,Count_in_last_14days,Count_in_last_28days
0,0,Balbriggan,2012-01-01,2012-01-01,0,0,0.0,9.7,8.3,6.7,...,1,1,0,0,0,2012-01-01,0,0,0,0
1,1,Balbriggan,2012-01-01,2012-01-01,1,0,0.0,9.6,8.2,6.6,...,1,1,0,0,0,2012-01-01,1,0,0,0
2,2,Balbriggan,2012-01-01,2012-01-01,2,0,0.0,9.1,7.7,6.0,...,1,1,0,0,0,2012-01-01,2,0,0,0
3,3,Balbriggan,2012-01-01,2012-01-01,3,0,0.0,9.1,7.5,5.6,...,1,1,0,0,0,2012-01-01,3,0,0,0
4,4,Balbriggan,2012-01-01,2012-01-01,4,0,0.0,8.5,7.3,5.8,...,1,1,0,0,0,2012-01-01,4,0,0,0


In [39]:
df.columns

Index(['Station Area', 'Date', 'Time Of Call', 'Count', 'rain', 'temp', 'wetb',
       'dewpt', 'vappr', 'rhum', 'msl', 'wdsp', 'wddir', 'ww', 'w', 'sun',
       'vis', 'clht', 'clamt', 'day_of_week', 'month', 'weekend', 'season',
       'Public Holiday', 'Count_in_last_1hour', 'Count_in_last_3hours',
       'Count_in_last_6hours', 'Count_in_last_7days', 'Count_in_last_14days',
       'Count_in_last_28days'],
      dtype='object')

In [38]:
df.to_csv('Final_full_ambulance_df.csv')